## 11-1 시군구별 인구 단계 구분도 만들기
- 지역별 통계치를 색깔 차이로 표현한 지도를 단계 구분도(choropleth map)이라 부른다
- 단계 구분도를 만들면 인구나 소득과 같은 통계치가 지역별로 어떻게 다른지 쉽게 알 수 있음

## Lab 1 : 시군구별 인구 단계 구분도 만들기

1. 시군구 경계 지도 데이터 준비하기
- 대한민국 시군구별 경계 좌표가 들어 있는 SIG.geojson 파일을 활용
- 행정구역 코드, 지역 이름, 시군구 경계 위도와 경도 좌표
- geojson은 위치 정보를 JSON 포맷으로 저장한 표준 지리 정보 데이터 포맷
- (주)지오서비스에서 공개한 SHP 파일을 오픈 소스 지리정보 시스템 QGIS로 변환하여 생성한 파일
- bit.ly/easypy_111

In [ ]:
import json
geo = json.load(open('SIG.geojson', encoding = 'UTF-8'))

- geo는 딕셔너리 자료 구조로 되어 있음
- geo의  properties에 들어 잇는 SIG_CD에 지역을 나타내는 행정구역 코드가 담김
- geometry에는 시군구의 경계를 나타내는 위도, 경도 좌표가 있음

In [ ]:
# 행정 구역 코드 출력
geo['features'][0]['properties']

In [ ]:
# 위도 경도 좌표 출력
geo['features'][0]['geometry']

2. 시군구별 인구 데이터 준비하기
- Population_SIG.csv : 2021년 시군구별 인구 데이터

In [ ]:
import pandas as pd
df_pop = pd.read_csv('Population_SIG.csv')
df_pop.head()

In [ ]:
df_pop.info()

In [ ]:
# code를 int64 타입에서 문자 타입으로 변경
df_pop['code'] = df_pop['code'].astype(str)

In [ ]:
df_pop.info()

3. 단계 구분도 만들기

In [ ]:
# folium 패키지 인스톨
# !pip install folium

> (1) 배경지도 만들기

In [ ]:
import folium

In [ ]:
# 배경지도 만들기 - 중심 위경도(35.95, 127.7), 확대 단계 8
folium.Map(location = [35.95, 127.7],  # 지도 중심 좌표
          zoom_start = 8,              # 확대 단계
          width = '50%',               # 지도 가로 크기
          heigth = '50%')              # 지도 세로 크기

In [ ]:
# 배경지도 만들기 - 중심 위경도(35.95, 127.7), 확대 단계 8, 지도 종류 변경
map_sig = folium.Map(location = [35.95, 127.7],
                      zoom_start = 8, 
                     tiles='cartodbpositron') # 지도 종류 - 단계 구분이 잘되는 밝은색 지도
map_sig

> (2) 단계 구분도 만들기
- folium.Choropleth()
    - geo_data : 지도 데이터
    - data : 색깔로 표현할 통계 데이터
    - columns :  통계 데이터의 행정 구역 코드 변수, 색깔로 표현할 변수
    - key_on : 지도 데이터의 행정구역 코드

In [ ]:
# 단계별 구분도 만들기
# 앞서 만든 배경지도 map_sig에 단계구분도를 덧씌움
# map_sig를 실행하면 시군구 경계가 표시된 지도가 출력
folium.Choropleth(
    geo_data=geo,                        # 지도 데이터
    data =df_pop,                        # 통계 데이터
    columns = ('code', 'pop'),           # df_pop 행정구역 코드, 인구
    key_on = 'feature.properties.SIG_CD').add_to(map_sig) # geo 행정구역 코드
map_sig

> 계급 구간 정하기

In [ ]:
# 인구를 5가지 계급 구간으로 나눔
# 값을 크기 순으로 나열한 다음 입력한 비율에 해당하는 값인 '분위수'를 구함
bins = list(df_pop["pop"].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
bins

> 디자인 수정하기

In [ ]:
# 배경지도 만들기 - 중심 위경도(35.95, 127.7), 확대 단계 8, 지도 종류 변경
map_sig = folium.Map(location = [35.95, 127.7],  # 지도 중심 위치
                    zoom_start = 8,              # 확대 단계
                     tiles='cartodbpositron')    # 지도 종류
map_sig

In [ ]:
# 단계별 구분도 만들기
folium.Choropleth(
    geo_data=geo,                      # 지도 데이터
    data =df_pop,                      # 통계 데이터
    columns = ('code', 'pop'),         # df_pop의 행정구역 코드, 인구
    key_on = 'feature.properties.SIG_CD', # geo 행정 구역 코드
    fill_color = 'YlGnBu',                # 컬러 맵
    fill_opacity = 1,                     # 투명도
    line_opacity = 0.5,                   # 경계선 투명도
    bins = bins).add_to(map_sig)          # 계급 구간 기준 값 / 배경 지도에 추가
map_sig

## 11-2 서울시 동별 외국인 인구 단계 구분도 만들기

## Lab 2 : 서울시 동별 외국인 인구 단계 구분도 만들기

1. 서울시 동 경계 지도 데이터 준비하기

In [ ]:
# 서울 동 경계 좌표가 들어있는 지도 데이터 불러오기
import json
geo_seoul = json.load(open('EMD_Seoul.geojson', encoding='UTF-8'))

In [ ]:
# 행정 구역 코드 출력
geo_seoul['features'][0]['properties']

In [ ]:
# 위도, 경도 좌표 출력
geo_seoul['features'][0]['geometry']

2. 서울시 동별 외국인 인구 데이터 준비 하기

In [ ]:
# 서울시 동별 외국인 통계 데이터 읽어오기
foreigner = pd.read_csv('Foreigner_EMD_Seoul.csv')
foreigner.head()

In [ ]:
foreigner.info()

In [ ]:
# 행정 구역 코드 타입을 문자로 변경
foreigner['code'] = foreigner['code'].astype(str)

3. 단계별 구분도 만들기

In [ ]:
#  외국인 수를 8가지 계급 구간으로 나눔
# 값을 크기 순으로 나열한 다음 입력한 비율에 해당하는 값인 '분위수'를 구함
bins = list(foreigner['pop'].quantile([0, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))
bins

In [ ]:
# 배경지도 만들기 - 서울 중심의 위경도(35.95, 127.7), 확대 단계 12, 지도 종류 변경
map_seoul = folium.Map(location = [37.56, 127],   # 서울 중심을 중앙에 배치
                        zoom_start = 12,          # 확대 단계 12
                       tiles='cartodbpositron')   # 지도 종류
map_seoul

In [ ]:
# 단계별 구분도 만들기
folium.Choropleth(
    geo_data=geo_seoul,                  # 서울 좌표
    data = foreigner,                    # 통계 데이터
    columns = ('code', 'pop'),           # 통계 데이터에서  행정구역 코드(code), 인구(pop) 표시
    key_on = 'feature.properties.ADM_DR_CD', # geo_seoul의 행정구역 코드
    fill_color = 'Blues',                   # 컬러맵
    nan_fill_color = 'White',                 # 결측치 색깔
    fill_opacity = 1,                       # 투명도
    line_opacity = 0.5,                     # 경게선 투명도
    bins = bins).add_to(map_seoul)       # 계급 구간 기준값 / 서울 배경지도에 추가
map_seoul

4. 구 경계선 추가하기

In [ ]:
# 구 경계 좌표를 담은 파일 읽어오기
geo_seoul_sig = json.load(open('SIG_Seoul.geojson', encoding='UTF-8'))

In [ ]:
# 서울 구 라인 추가
folium.Choropleth(
    geo_data=geo_seoul_sig,             #지도 데이터
    fill_opacity = 0,                   # 투명도
    line_weight = 4).add_to(map_seoul) # 선 두께 / 배경지도에 추가
map_seoul

- html로 저장하기

In [ ]:
# map을 html 파일로 저장, 웹브라우저로 확인 가능
map_seoul.save('map_seoul.html')

In [ ]:
# 웹브라우저 패키지의 open_new()를 이용하여 HTML 파일을 열기
import webbrowser
webbrowser.open_new('map_seoul.html')

- 다양한 지도 만들기
> python-visualization.github.io/folium